In [1]:
import networkx as nx

from burn_tree import *

In [23]:
def burn_most_leaves(tree):
    activators = []
    marked = set()
    
    i = 0
    while tree.order() > 0:
        print("\nTree size", tree.order())
        print("Nodes:", tree.nodes())
        print("Edges:", tree.edges())
        # Consider all vertices v of height at most i - which N_i[v] covers the most leaves?
        leaves = get_leaves(tree)
        node_distances = shortest_path_lengths(tree)
        
        # Get all vertices within distance i of a leaf
        near_leaves = set()
        for leaf in leaves:
            nhood = get_neighbourhood(tree, source=leaf, radius=i, node_distances=node_distances)
            for node in nhood:
                near_leaves.add(node)
        
        # For all v we just got: take one with max leaves in its neighbourhood
        max_leaves = 0
        max_node = None
        max_nhood = None
        for node in near_leaves:
            nhood = get_neighbourhood(tree, source=node, radius=i, node_distances=node_distances)
            
            num_leaves = 0
            for v in nhood:
                if v in leaves:
                    num_leaves += 1
            
            if num_leaves > max_leaves:
                max_node = node
                max_nhood = nhood
                max_leaves = num_leaves
            
        # Burn that vertex
        activators.insert(0, tree.nodes(data=True)[max_node])
        
        # Remove all vertices we can without disconnecting the graph
        eccens = nx.algorithms.distance_measures.eccentricity(tree)
        sorted_eccens = sorted(eccens.items(), key=lambda kv: kv[1], reverse=True)
        for node, eccen in sorted_eccens:
            # Only remove nodes within the appropriate neighbourhood
            if node in nhood:
                temp = tree.copy()
                temp.remove_node(node)
                
                # Only remove if the the tree does not become disconnected
                if temp.order() == 0 or nx.algorithms.components.is_connected(temp):
                    print("Removing", node)
                    tree = temp
                else:
                    print("Not removing", node)
        
        print(max_node)
        i += 1
    
    return activators

tree = nx.Graph()
for i in range(5):
    tree.add_node(i, key=i)
edges = [(0, 1), (0, 2), (0, 3), (0, 4)]
tree.add_edges_from(edges)
print(list(tree.nodes(data=True)))

bs = burn_most_leaves(tree)
print("\nBurning sequence", bs)

[(0, {'key': 0}), (1, {'key': 1}), (2, {'key': 2}), (3, {'key': 3}), (4, {'key': 4})]

Tree size 5
Nodes: [0, 1, 2, 3, 4]
Edges: [(0, 1), (0, 2), (0, 3), (0, 4)]
Removing 4
1

Tree size 4
Nodes: [0, 1, 2, 3]
Edges: [(0, 1), (0, 2), (0, 3)]
Removing 3
Not removing 0
0

Tree size 3
Nodes: [0, 1, 2]
Edges: [(0, 1), (0, 2)]
Removing 1
Removing 2
Removing 0
0

Burning sequence [{'key': 0}, {'key': 0}, {'key': 1}]
